In [ ]:
!pip install --upgrade xlrd

In [94]:
import numpy as np
import pandas as pd
import networkx
import sklearn.preprocessing
from scipy.sparse import csc_matrix

In [3]:
x = np.array([[0, 2], [0, 0], [2,3]])

sklearn.preprocessing.normalize(x, norm="l1", axis=0, copy=False)
x

array([[0, 2],
       [0, 0],
       [2, 3]])

In [4]:
def Normalize(M):
  return sklearn.preprocessing.normalize(M, norm="l1", axis=0, copy=False)

In [89]:
def RegularizationMatrix(M, Mg, b):
  n = M.shape[0]
  M1 = np.ones((n,1))
  m = M @ M1
  p = M.transpose() @ m
  p = p.transpose()
  P = np.diag(p.flatten())
  print(p.shape)
  print(P.shape)
  #Mr = Mg @ np.linalg.matrix_power(P, -b)
  Mr = Mg
  Mr = Normalize(M)
  return Mr

In [6]:
def Inflate(M, r, count, beta):
  n = M.shape[0]
  for i in range(n):
    p = r * (beta ** count[i])
    M[i,:] = M[i,:] ** p
  M = Normalize(M)
  return M

In [61]:
def Prune(M, threshold = 0.0001):
  #print(threshold)
  #print(M[0,0]<threshold)
  #print(M[0,0])
  M[M < threshold] = 0
  M = Normalize(M)
  return M

In [8]:
def CheckConverged(M0,M1):
  return np.allclose(M0, M1)

In [9]:
def attractors(M):
  return np.unique(np.nonzero(M)[0])

In [60]:
def clusters(M):
  cluster = set()
  attractor = attractors(M)
  for a in attractor:
    #print(M[a,:])
    #print(np.nonzero(M[a,:])[0])
    cluster.add(tuple(np.nonzero(M[a,:])[0].tolist()))
  return cluster

In [11]:
def NA(C1, C2):
  inter = len(tuple(set(C1) & set(C2)))
  return (inter**2)/(len(C1) * len(C2))

In [71]:
def qf(C1):
  global A
  k = len(C1)
  ecount = 0
  for i in range(k):
    for j in range(i + 1, k):
      if(A[i,j] != 0):
        ecount += 1
  return ecount / (k * (k-1))

In [66]:
def AddLoops(A):
  n = A.shape[0]
  for i in range(n):
    A[i, i] = max(A[i])
  return A

In [37]:
x = np.array([[0,3,5],[1,0,2],[10,20,0]])

x = AddLoops(x)
print(x)

[[ 5  3  5]
 [ 1  2  2]
 [10 20 20]]


In [109]:
y = np.zeros(10)
z = np.array([2,5,6])
y[z] += 1
print(y)

[0. 0. 1. 0. 0. 1. 1. 0. 0. 0.]


In [98]:
def SRMCL(Mg, b, r, beta, t):
  C = set()
  n = Mg.shape[0]
  count = np.zeros(n)
  M = np.copy(Mg)

  for i in range(t):
    while(True):
      lastM = np.copy(M)
      Mr = RegularizationMatrix(M, Mg, b)
      M = M @ Mr
      M = Inflate(M, r, count, beta)
      M = Prune(M)
      print(len(np.nonzero(M)[0]))
      if(CheckConverged(M, lastM)):
        break
    Titer = attractors(M)
    count[Titer] += 1
    Citer = clusters(M)
    #return M, Titer, Citer
    C = C.union(Citer)
    return C, M, Titer, Citer
  return C

    

In [72]:
def post_processing(C, qf, w, p):
  C = np.array(list(C))
  k = len(C)
  dl = []
  for i in range(k):
    if(qf[C[i]] < w or len(C[i]) <= 2):
      dl.append(i)
  dl = np.array(dl)
  C = np.delete(C, dl)
  Cqf = np.array(list(map(qf, C)))
  Cindex = Cqf.argsort()
  C = C[Cindex[::-1]]
  k = len(C)
  dl = []
  for i in range(k):
    for j in range(i + 1, k):
      if(NA(C[i], C[j]) >= p):
        dl.append(j)

  dl = np.array(dl)
  dl = np.unique(dl)
  C = np.delete(C, dl)
  
  return C
  

In [38]:
df = pd.read_excel("/content/pro200600448_1_s.xls").drop(["Data sets", "# supporting data sets"], axis = 1)
names = pd.concat([df["Protein A"], df["Protein B"]]).unique()
n = len(names)
hash = dict(zip(names, range(n)))
A = np.zeros((n, n))
graph = networkx.from_numpy_matrix(A)

for index, row in df.iterrows():
  i = hash[row["Protein A"]]
  j = hash[row["Protein B"]]
  A[i,j] = row["Score"]
  A[j,i] = row["Score"]

In [39]:
A = AddLoops(A)
MG = Normalize(A)

In [55]:
x = np.array([1,2,4,5])
print(np.diag(x))

[[1 0 0 0]
 [0 2 0 0]
 [0 0 4 0]
 [0 0 0 5]]


In [99]:
Mg = np.copy(MG)
cluster, M,titer,citer = SRMCL(Mg, 2, 2, 1.25, 1)
#cluster = SRMCL(Mg, 2, 2, 1.25, 1)

(1, 3034)
(3034, 3034)
94545
(1, 3034)
(3034, 3034)
54408
(1, 3034)
(3034, 3034)
27137
(1, 3034)
(3034, 3034)
14229
(1, 3034)
(3034, 3034)
8160
(1, 3034)
(3034, 3034)
5412
(1, 3034)
(3034, 3034)
4285
(1, 3034)
(3034, 3034)
3667
(1, 3034)
(3034, 3034)
3415
(1, 3034)
(3034, 3034)
3328
(1, 3034)
(3034, 3034)
3201
(1, 3034)
(3034, 3034)
3189
(1, 3034)
(3034, 3034)
3182
(1, 3034)
(3034, 3034)
3181
(1, 3034)
(3034, 3034)
3181


In [101]:
print(len(cluster))
print(len(citer))

617
617


In [73]:
clustercopy = np.copy(cluster)